In [41]:
import os
import pandas as pd
import yfinance as yf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [42]:
def load_process_data(ticker, start_date, end_date, dealNaN=True, drop_NaN=True, split_by_ratio=True, split_by_date=False, split_by_randomly=False, split_date=None, scale=True):
    DATA_DIR = 'stock_data'
    
    test_size = 0.2
    feature_columns = ["Open","High","Low","Close","Adj Close"]
    
    # Check if the data directory exists, if not, create it
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
    
    # Create a file path for the stock data
    file_path = os.path.join(DATA_DIR, f'{ticker}_{start_date}_{end_date}.csv')
    
    # Load or download data
    if os.path.exists(file_path):
        print(f"Loading data from {file_path}")
        data = pd.read_csv(file_path, index_col='Date', parse_dates=True)
    else:
        print(f"Downloading data for {ticker} from Yahoo finance")
        data = yf.download(ticker, start=start_date, end=end_date)
        data.to_csv(file_path)
        print(f"Data saved to {file_path}")
    
    result = {'data': data.copy()}
    
    # Handle NaN values
    if dealNaN:
        # Check for NaN values before handling
        print("NaN values before handling")
        print(data.isna().sum())
        # Depends on chosen drop or fill NaN
        if drop_NaN is True:
            data.dropna(inplace=True)
        else:
            data.fillna(data.mean(), inplace=True)
        # Check for NaN values after handling
        print("NaN values after handling")
        print(data.isna().sum())
    
    # Split data with specified method
    if split_by_ratio:
        print('Splitting by ratio')
        train_samples = int((1-test_size) * len(data))
        
        result['data_train'] = data[:train_samples]
        result['data_test'] = data[train_samples:]
        
        print(f"Training samples: {len(result['data_train'])}")
        print(f"Test samples: {len(result['data_test'])}")
        
    elif split_by_date and split_date is not None:
        print('Splitting by date')
        split_date = pd.to_datetime(split_date)
        
        result['data_train'] = data[data.index <= split_date]
        result['data_test'] = data[data.index > split_date]
        
        print(f"Training samples: {len(result['data_train'])}")
        print(f"Test samples: {len(result['data_test'])}")
    elif split_by_randomly:
        print('Randomly splitting with shuffling')
        result['data_train'], result['data_test'] = train_test_split(data, test_size=test_size, shuffle=True)
        
        print(f"Training samples: {len(result['data_train'])}")
        print(f"Test samples: {len(result['data_test'])}")
    
    # Optionally scale feature columns
    if scale:
        scaler = MinMaxScaler(feature_range=(0,1))
        data[feature_columns] = scaler.fit_transform(data[feature_columns])
        result['scaled_data'] = data[feature_columns]
        
    return result

In [43]:
TICKER = 'AAPL'
START_DATE = '2020-01-01'
END_DATE = '2023-01-01'

result = load_process_data(ticker=TICKER, start_date=START_DATE, end_date=END_DATE, split_by_date=False, split_by_randomly=False, split_by_ratio=True, split_date='06-01-2021', scale=True)

print("Train data in first 5 rows")
print(result['data_train'].head())
print("Test data in first 5 rows")
print(result['data_test'].head())

[*********************100%%**********************]  1 of 1 completed

Data saved to stock_data/AAPL_2020-01-01_2023-01-01.csv
NaN values before handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
NaN values after handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Splitting by ratio
Training samples: 604
Training samples: 152
Train data in first 5 rows
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2020-01-02  74.059998  75.150002  73.797501  75.087502  72.796028  135480400
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.088287  146322800
2020-01-06  73.447502  74.989998  73.187500  74.949997  72.662704  118387200
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.320969  108872000
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.484344  132079200
Test data in first 5 rows
                  Open        

In [44]:
result = load_process_data(ticker=TICKER, start_date=START_DATE, end_date=END_DATE, split_by_date=True, split_by_randomly=False, split_by_ratio=False, split_date='06-01-2021', scale=True)

Loading data from stock_data/AAPL_2020-01-01_2023-01-01.csv
NaN values before handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
NaN values after handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Splitting by date
Training samples: 356
Training samples: 400


In [45]:
result = load_process_data(ticker=TICKER, start_date=START_DATE, end_date=END_DATE, split_by_date=False, split_by_randomly=True, split_by_ratio=False, split_date='06-01-2021', scale=True)

print("Train data in first 5 rows")
print(result['data_train'].head())
print("Test data in first 5 rows")
print(result['data_test'].head())

Loading data from stock_data/AAPL_2020-01-01_2023-01-01.csv
NaN values before handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
NaN values after handling
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Randomly splitting with shuffling
Training samples: 604
Training samples: 152
Train data in first 5 rows
                  Open        High  ...   Adj Close     Volume
Date                                ...                       
2022-04-07  171.160004  173.360001  ...  169.581436   77594700
2022-01-28  165.710007  170.350006  ...  167.584839  179935700
2021-03-05  120.980003  121.940002  ...  118.908562  153766600
2021-07-19  143.750000  144.070007  ...  139.740540  121434600
2022-10-07  142.539993  143.100006  ...  138.402542   85925600

[5 rows x 6 columns]
Test data in first 5 rows
                  Open        High  ...   Adj Close     Volume
Date                    